# Sprawozdanie z labratorium 8 - Kacper Barczyk - 11.05.2021

In [30]:
import mpmath as mp
from mpmath import sin
from mpmath import cos
from mpmath import sinh
from mpmath import cosh
from mpmath import tan
from mpmath import exp
from mpmath import mpf
from math import pi
from mpmath import sign
import colorsys
import matplotlib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
from math import ceil
from math import floor
from math import log
from numpy import float64
import random
import time
import scipy
from scipy.linalg import lu
from itertools import product
from mpl_toolkits.mplot3d import Axes3D
from numpy import random
from numpy import allclose
import networkx as nx

# Zad 1

In [31]:
def powerMethod(matrix, epsilon, maxIterations):
    (n, _) = matrix.shape
    
    currentV = np.random.rand(n)
    nextV = matrix @ currentV 
    maxValue = np.linalg.norm(nextV, ord=np.inf)
    nextV /= maxValue
    
    i = 0
    while i < maxIterations and np.linalg.norm(currentV - nextV) > epsilon:
        currentV = nextV
        nextV = matrix @ currentV
        maxValue = np.linalg.norm(nextV, ord=np.inf)
        nextV /= maxValue
        i += 1
    
    nextV /= np.linalg.norm(nextV)
    
    return (maxValue, nextV)

In [32]:
n = 5
matrix = np.random.rand(n, n)
epsilon = 10 ** (-4)
maxIterations = 1000

(maxEigenvalue, eigenvector) = powerMethod(matrix, epsilon, maxIterations)
npEigenvalue = max(np.linalg.eig(matrix)[0])
npEigenvector = np.linalg.eig(matrix)[1][:, 0]

print("Najwieksza wartość własna wyznaczona za pomocą powerMathod:", maxEigenvalue)
print("Najwieksza wartość własna wyznaczona za pomocą funkcji bibliotecznej:", npEigenvalue)
print("-----------------------")
print("Wektor własny wyznaczony za pomocą powerMathod:", eigenvector)
print("Wektor własny wyznaczony za pomocą funkcji bibliotecznej:", eigenvector)

Najwieksza wartość własna wyznaczona za pomocą powerMathod: 2.6122074886667663
Najwieksza wartość własna wyznaczona za pomocą funkcji bibliotecznej: (2.6122350841691366+0j)
-----------------------
Wektor własny wyznaczony za pomocą powerMathod: [0.43092412 0.37362182 0.58068444 0.37986959 0.43956321]
Wektor własny wyznaczony za pomocą funkcji bibliotecznej: [0.43092412 0.37362182 0.58068444 0.37986959 0.43956321]


In [ ]:
epsilon = 10 ** (-8)
maxIterations = 10000

timesPowerMethod = []
for i in range(100, 2500, 100):
    start = time.process_time()
    n = i
    matrix = np.random.rand(n, n)
    (maxEigenvalue, eigenvector) = powerMethod(matrix, epsilon, maxIterations)
    end = time.process_time()
    timesPowerMethod.append(end - start)

timesNumpyMethod = []
for i in range(100, 2500, 100):
    start = time.process_time()
    n = i
    matrix = np.random.rand(n, n)
    np.linalg.eig(matrix)
    end = time.process_time()
    timesNumpyMethod.append(end - start)

In [ ]:
sizesOfMatrices = []
for i in range(100, 2500, 100):
    sizesOfMatrices.append(i)

plt.figure(figsize=(10, 8))
plt.scatter(sizesOfMatrices, timesPowerMethod, label='power method')
plt.scatter(sizesOfMatrices, timesNumpyMethod, label='np.linalg.eig function')
plt.xlabel('rozmiar macierzy', fontsize=10)
plt.ylabel('czas działania [s]', fontsize=10)
plt.legend(prop={'size': 10})
plt.title("Porównanie czasów działania metod power method i bibliotecznej w zależności od rozmiaru macierzy")

Jak widać na podstawie wykresu, zaimplementowana funkcja powerMethod jest znacznie szybsza od funcji bibliotecznej. Wynika to z faktu, iż funkcja biblioteczna zwraca wszystkie wartości własne i wszystkie odpowiadające wektory własne, natomiast moja funkcja zwraca tylko jedną, największą wartość własną i odpowiadający jej jeden wektor.

# Zad 2

Korzystając z twierdzenia podanego w zadaniu, mówiącego o identyczności wektorów własnych macierzy i macierzy do niej odwrotnej (z przesunięciem) będę wyznaczał wektor własny wykorzystując metodę potęgową na macierzy odwrotnej oraz parametr wejściowy sigma. Będę wykonywał następujące kroki:

$$x^{k+1} = (A- \sigma I  )^{-1} x^k$$

Przekształcam do równoważnej postaci łatwiejszej do obliczenia

$$A x^{k+1} = x^k$$

Przekształcam do równoważnej postaci wykorzystującej faktoryzację LU

$$U x^{k+1} = L^{-1} x^k$$

Rozwiązuję układ równań gdzie niewiadomomą jest wektor $$ x^{k+1} $$

i normalizuję rozwiązanie. Powtarzam czynności aż do uzyskania żądanej dokładności, lub do uzyskania odpowiedniej liczby iteracji

In [ ]:
def inversePowerMethod(matrix, epsilon, maxIterations, sigma):
    (n, _) = matrix.shape
    I = np.identity(n)
    shiftMatrix = matrix.copy()
    
    shiftMatrix = shiftMatrix - sigma * I
    _, L, U = scipy.linalg.lu(shiftMatrix)
    invL = np.linalg.inv(L)
        
    currentV = np.random.rand(n)
    rightSide = invL @ currentV
    nextV = np.linalg.solve(U, rightSide)
    maxValue = np.linalg.norm(nextV)
    nextV /= maxValue
    
    i = 0
    while i < maxIterations and np.linalg.norm(currentV - nextV) > epsilon:
        currentV = nextV
        rightSide = invL @ currentV
        nextV = np.linalg.solve(U, rightSide)
        maxValue = np.linalg.norm(nextV)
        nextV /= maxValue
        i += 1
    
    nextV /= np.linalg.norm(nextV)
    
    return nextV, i

In [ ]:
n = 3
matrix = np.random.rand(n, n)
epsilon = 10 ** (-8)
maxIterations = 2000

eigenvector, iterations = inversePowerMethod(matrix, epsilon, maxIterations, 2)
npEigenvector = np.linalg.eig(matrix)[1][:, 0]

print("Wektor własny wyznaczony za pomocą inversePowerMethod:", eigenvector)
print("Wektor własny wyznaczony za pomocą funkcji bibliotecznej:", npEigenvector)
print("Liczba iteracji potrzebna do znalezienia rozwiązania:", iterations)

Metoda znajduje poprawny wektor własny, ale potrzebuje do tego dużej liczby iteracji (maksymalnej zadanej jako parametr).

# Zad 3

In [ ]:
def ipmWithRayleighQuotient(matrix, epsilon, maxIterations, sigma):
    (n, _) = matrix.shape
    I = np.identity(n)
    shiftMatrix = matrix.copy()
    
    shiftMatrix = shiftMatrix - sigma * I
    _, L, U = scipy.linalg.lu(shiftMatrix)
    invL = np.linalg.inv(L)
    
    currentV = np.random.rand(n)
    rightSide = invL @ currentV
    nextV = np.linalg.solve(U, rightSide)
    maxValue = np.linalg.norm(nextV)
    nextV /= maxValue
    
    i = 0
    while i < maxIterations and np.linalg.norm(currentV - nextV) > epsilon:
        
        sigma = ( currentV.T @ shiftMatrix @ currentV ) / ( currentV.T @ currentV )
        shiftMatrix = shiftMatrix - sigma * I
        _, L, U = scipy.linalg.lu(shiftMatrix)
        invL = np.linalg.inv(L)
        
        currentV = nextV
        rightSide = invL @ currentV
        nextV = np.linalg.solve(U, rightSide)
        maxValue = np.linalg.norm(nextV)
        nextV /= maxValue
        i += 1
    
    nextV /= np.linalg.norm(nextV)
    
    return nextV, i

In [ ]:
n = 3
matrix = np.random.rand(n, n)
epsilon = 10 ** (-8)
maxIterations = 2000

eigenvector, iteration = ipmWithRayleighQuotient(matrix, epsilon, maxIterations, 2)
npEigenvalue = min(np.linalg.eig(matrix)[0])
npEigenvector = np.linalg.eig(matrix)[1][:, 0]

print("Wektor własny wyznaczony za pomocą ipmWithRayleighQuotient:", eigenvector)
print("Wektor własny wyznaczony za pomocą funkcji bibliotecznej:", npEigenvector)
print("Liczba iteracji potrzebna do znalezienia rozwiązania:", iterations)

Metoda znajduje poprawny wektor własny, i w stosunku do zwykłej odwrotnej metody potęgowej potrzebuje znacznie mniejszej ilości iteracji (przetestowane wielokrotnie). Metoda z ilorazem Rayleigha ma jednak większą złożoność. 